# Recomendación de tarifas

Usted tiene a su disposición datos sobre el comportamiento de los clientes que ya han pasado en estas tarifas (del proyecto del curso «Análisis de datos estadísticos»). Es necesario construir un modelo para la tarea de clasificación que elegirá una tasa adecuada. Usted no necesita pre-procesar los datos - ya lo ha hecho.

Construya un modelo con el valor más alto de *precisión*. Para aprobar el proyecto con éxito, debe llevar la proporción de respuestas correctas a al menos 0,75. Compruebe usted mismo la *exactitud* de la muestra de ensayo.

## Abrir y examinar el archivo

In [1]:
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import pandas as pd

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Abra el archivo y asegúrese de que los datos son realmente buenos. Tenemos 5 características: Número de llamadas, Número total de minutos, Número de mensajes, Tráfico usado y Tarifa.
Hay 3214 filas en la matriz de datos, sin espacios en blanco.
Las funciones Número de llamadas y Número de mensajes solo contienen enteros. Puede cambiar el tipo de datos a int.

In [3]:
for column in df[['calls','messages']]:
    df[column] = df[column].astype("int")

También, por si acaso, revisa los datos para ver si hay duplicados obvios.

In [4]:
df.duplicated().sum()

0

No hay duplicados obvios. Ahora definitivamente puedes empezar a trabajar.

## Dividir datos en muestras

Divida los datos iniciales en muestras de entrenamiento, validación y prueba en proporciones 3:1:1.

In [5]:
features = df.drop(['is_ultra'], axis=1) 
target = df['is_ultra'] 
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.4, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(features_test, target_test, test_size=0.5, random_state=12345) 
print(features_train.shape)
print(target_train.shape)
print(features_test.shape)
print(target_test.shape)
print(features_valid.shape)
print(target_valid.shape)

(1928, 4)
(1928,)
(643, 4)
(643,)
(643, 4)
(643,)


El porcentaje es correcto, podemos empezar la investigación.

## Explorar los modelos

Exploramos tres modelos de aprendizaje automático: Decision Tree Classifier, RandomForestClassifier y Logistic Regression.

### Decision Tree Classifier

In [6]:
best_model = None
best_depth = 0
best_result = 0
for depth in range(1, 11):
	model = DecisionTreeClassifier(random_state=12345, max_depth=depth) # обучаем модель с заданной глубиной дерева
	model.fit(features_train, target_train) # обучаем модель
	predictions_valid = model.predict(features_valid) # получаем предсказания модели
	result = accuracy_score(predictions_valid, target_valid) # посчитаем качество модели
	if result > best_result:
		best_model = model
		best_depth = depth
		best_result = result
        
print('best max_depth =', 
      best_depth, 
      '; ', 
      'best Accuracy = ', 
      best_result, 
      end =''
     ) 

best max_depth = 3 ;  best Accuracy =  0.7853810264385692

El mejor hiperparámetro de profundidad de árbol es 3 en el modelo "Descifrar árbol", la tasa de respuesta correcta es 0.785

### RandomForestClassifier

In [12]:
best_model = None
best_result = 0
best_est = 0
best_depth = 0

for est in range(10, 21):
    for depth in range(1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = depth) 
        model.fit(features_train, target_train) 
        result = model.score(features_valid, target_valid) 
        if result > best_result:
            best_model = model 
            best_result = result 
            best_est = est
            best_depth = depth
    
print('best n_estimators =', 
      best_est, 
      '; ',
      'best max_depth =', 
      best_depth, 
      '; ', 
      'best Accuracy = ', 
      best_result, 
      end =''
     ) 

best n_estimators = 12 ;  best max_depth = 6 ;  best Accuracy =  0.8040435458786936

En el modelo "Random Forest" obtener los mejores hiperparámetros: para el número de árboles - 8 y para la profundidad del árbol - 8. La proporción de respuestas correctas será 0.802 - por el momento es el mejor resultado.

### Logistic Regression

In [8]:
model = LogisticRegression(random_state=12345) 
model.fit(features_train, target_train) 
predictions_valid = model.predict(features_valid)
score = accuracy_score(predictions_valid, target_valid)
print('Accuracy LogisticRegression:', score, end ='')

Accuracy LogisticRegression: 0.7107309486780715

En el modelo de "Regresión Logística", la tasa de respuesta es 0,711, que es menor que en los dos modelos anteriores.

## Compruebe el modelo en la muestra de prueba

El mejor resultado mostró el modelo "Random Forest", por lo que lo probaremos en una muestra de prueba.

In [13]:
model = RandomForestClassifier(random_state=12345, n_estimators = 12, max_depth = 6)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
result_test = accuracy_score(predictions_test, target_test)
print('Accuracy RandomForestClassifier:', result_test)

Accuracy RandomForestClassifier: 0.7947122861586314


El modelo mostró buenos resultados en la muestra de prueba, la tasa de respuesta correcta cambió muy poco - 0,796.

## (bonus) Comprueba si los modelos son adecuados

Compruebe la adecuación de nuestro modelo con el modelo DummyClassifier más simple, que siempre predice la clase más común. Así que tenemos los datos de referencia para comparar el modelo que hemos construido. Además, podremos estar seguros de la calidad de su trabajo, por ejemplo, que produce no solo un conjunto de datos seleccionados al azar.

In [10]:
model_dummy = DummyClassifier(strategy='most_frequent', random_state=12345)
model_dummy.fit(features_train, target_train)
result_dummy = model_dummy.score(features_valid, target_valid)
print('Accuracy DummyClassifier:', result_dummy)
print('Accuracy RandomForestClassifier:', result_test)

Accuracy DummyClassifier: 0.7060653188180405
Accuracy RandomForestClassifier: 0.7962674961119751


La tasa de respuestas correctas en el modelo más simple es mucho menor, por lo que nuestro modelo es adecuado y eficaz.

## Conclusión general

Los datos fueron proporcionados correctamente, el pre-procesamiento como tal no era necesario.
Se investigaron tres modelos: Decision Tree Classifier, RandomForestClassifier y Logistic Regression.
El resultado más bajo fue la Regresión Logística con una puntuación de 0,711.
El mejor resultado mostró el modelo RandomForestClassifier con hiperparámetros n_estimators iguales a 8 y máx. La tasa de respuesta correcta fue de 0,802. En la muestra de prueba el resultado fue de 0,796. 
El DummyClassifier se utilizó en la prueba de adecuación, con un resultado de 0,706, que demostró que el RandomForestClassifier era adecuado.